### МОДЕЛИ КЛАССИФИКАЦИИ ДЛЯ ПРОГНОЗИРОВАНИЯ ОТКЛИКА

In [ ]:
# Импорт необходимых библиотек для классификации
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_auc_score, confusion_matrix, classification_report,
                             roc_curve, precision_recall_curve)
import joblib
import pickle
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from collections import Counter

In [ ]:
df = pd.read_csv('../data/marketing_campaign.csv', sep='\t')

# Нормализация категорий
df["Education"] = df["Education"].replace({"2n Cycle": "Pre-Graduate", "Basic": "Pre-Graduate"})
df["Marital_Status"] = df["Marital_Status"].replace({
    "Married": "Married/Together", "Together": "Married/Together",
    "Single": "Single", "Divorced": "Other", "Widow": "Other",
    "Alone": "Other", "Absurd": "Other", "YOLO": "Other"
})

# Feature engineering
df["Kids"] = df["Kidhome"].astype("int8") + df["Teenhome"].astype("int8")
df["Expenses"] = df[["MntWines", "MntFruits", "MntMeatProducts", "MntFishProducts", "MntSweetProducts", "MntGoldProds"]].sum(axis=1)
df["TotalAcceptedCmp"] = df[["AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5"]].astype("int8").sum(axis=1)
df["TotalNumPurchases"] = df[["NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases", "NumDealsPurchases"]].sum(axis=1)

# Удаление лишних колонок
df.drop(columns=["Kidhome", "Teenhome", "MntWines", "MntFruits", "MntMeatProducts", "MntFishProducts", 
                "MntSweetProducts", "MntGoldProds", "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", 
                "AcceptedCmp4", "AcceptedCmp5", "NumWebPurchases", "NumCatalogPurchases", 
                "NumStorePurchases", "NumDealsPurchases"], inplace=True)

df["Kids"] = df["Kids"].replace({0: "No Kid", 1: "Has Kids", 2: "Has Kids", 3: "Has Kids"})
df["TotalAcceptedCmp"] = df["TotalAcceptedCmp"].replace({0: "0", 1: ">0", 2: ">0", 3: ">0", 4: ">0"})

num_features = ["Income", "Recency", "NumWebVisitsMonth", "Expenses", "TotalNumPurchases"]
cat_features = ["Education", "Marital_Status", "Response", "Complain", "Kids", "TotalAcceptedCmp"]

# Очистка Income
df['Income'] = df['Income'].fillna(df['Income'].median())
df = df[df['Income'] < 600000]

In [ ]:
# Определяем признаки для модели
# Числовые признаки 
numeric_features = ["Income", "Recency", "NumWebVisitsMonth", "Expenses", "TotalNumPurchases"]

# Категориальные признаки 
categorical_features = ["Education", "Marital_Status", "Complain", "Kids", "TotalAcceptedCmp"]

# Выбираем все признаки для модели
feature_cols = numeric_features + categorical_features 

In [ ]:
# Создаем матрицу признаков и целевую переменную
X = df[feature_cols].copy()
y = df['Response'].copy()

In [ ]:
# Визуализация дисбаланса классов
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# Pie chart
axes[0].pie(y.value_counts(), labels=['Нет отклика (0)', 'Отклик (1)'], 
            autopct='%1.1f%%', colors=['#ff9999', '#66b3ff'], startangle=90)
axes[0].set_title('Распределение целевой переменной', fontsize=14)
# Bar chart
axes[1].bar(['Нет отклика (0)', 'Отклик (1)'], y.value_counts(), 
            color=['#ff9999', '#66b3ff'], edgecolor='black')
axes[1].set_title('Количество наблюдений по классам', fontsize=14)
axes[1].set_ylabel('Количество')

plt.tight_layout()
plt.show()

In [ ]:
# Подготовка данных для классификации
# Целевая переменная - Response (отклик на последнюю кампанию)
print("Распределение целевой переменной:")
print(df['Response'].value_counts())

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

In [ ]:
# Создаем препроцессор для обработки разных типов данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ]
)

# Обучаем препроцессор на тренировочных данных
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Получаем названия признаков после обработки
# Для числовых - оставляем оригинальные названия
num_feature_names = numeric_features

# Для категориальных - получаем названия после one-hot
cat_feature_names = []
for cat_feat in categorical_features:
    categories = preprocessor.named_transformers_['cat'].categories_[categorical_features.index(cat_feat)]
    for cat in categories:
        cat_feature_names.append(f"{cat_feat}_{cat}")
        
all_feature_names = num_feature_names + cat_feature_names

print(f"Всего признаков после обработки: {len(all_feature_names)}")

# Сохраняем препроцессор
joblib.dump(preprocessor, 'preprocessor.pkl')

### МЕТОД 1: ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ

In [ ]:
# Логистическая регрессия с подбором гиперпараметров
log_reg_params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear'],
    'class_weight': [None, 'balanced']
}

log_reg = LogisticRegression(random_state=42, max_iter=1000)

# Grid Search с кросс-валидацией
log_reg_grid = GridSearchCV(
    log_reg, 
    log_reg_params, 
    cv=StratifiedKFold(5, shuffle=True, random_state=42),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

log_reg_grid.fit(X_train_processed, y_train)

print("\nЛучшие параметры для логистической регрессии:")
print(log_reg_grid.best_params_)
print(f"Лучший ROC-AUC на кросс-валидации: {log_reg_grid.best_score_:.4f}")

# Лучшая модель
best_log_reg = log_reg_grid.best_estimator_

# Предсказания
y_pred_log_reg = best_log_reg.predict(X_test_processed)
y_pred_proba_log_reg = best_log_reg.predict_proba(X_test_processed)[:, 1]

In [ ]:

# Метрики для логистической регрессии
print("\n" + "="*60)
print("МЕТРИКИ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ")
print("="*60)

log_reg_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_log_reg),
    'Precision': precision_score(y_test, y_pred_log_reg),
    'Recall': recall_score(y_test, y_pred_log_reg),
    'F1-Score': f1_score(y_test, y_pred_log_reg),
    'ROC-AUC': roc_auc_score(y_test, y_pred_proba_log_reg)
}

for metric, value in log_reg_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_log_reg, target_names=['Нет отклика', 'Отклик']))

# Матрица ошибок
cm_log_reg = confusion_matrix(y_test, y_pred_log_reg)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_log_reg, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Нет отклика', 'Отклик'],
            yticklabels=['Нет отклика', 'Отклик'])
plt.title('Матрица ошибок - Логистическая регрессия', fontsize=14)
plt.ylabel('Фактическое значение')
plt.xlabel('Предсказанное значение')
plt.show()


### МЕТОД 2: СЛУЧАЙНЫЙ ЛЕС (Random Forest)

In [ ]:
# Random Forest с подбором гиперпараметров
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': [None, 'balanced']
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Grid Search с кросс-валидацией
rf_grid = GridSearchCV(
    rf, 
    rf_params, 
    cv=StratifiedKFold(5, shuffle=True, random_state=42),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train_processed, y_train)

print("\nЛучшие параметры для Random Forest:")
print(rf_grid.best_params_)
print(f"Лучший ROC-AUC на кросс-валидации: {rf_grid.best_score_:.4f}")

# Лучшая модель
best_rf = rf_grid.best_estimator_

# Предсказания
y_pred_rf = best_rf.predict(X_test_processed)
y_pred_proba_rf = best_rf.predict_proba(X_test_processed)[:, 1]

In [ ]:
# Метрики для Random Forest
print("\n" + "="*60)
print("МЕТРИКИ RANDOM FOREST")
print("="*60)

rf_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_rf),
    'Precision': precision_score(y_test, y_pred_rf),
    'Recall': recall_score(y_test, y_pred_rf),
    'F1-Score': f1_score(y_test, y_pred_rf),
    'ROC-AUC': roc_auc_score(y_test, y_pred_proba_rf)
}

for metric, value in rf_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Нет отклика', 'Отклик']))

# Матрица ошибок
cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Нет отклика', 'Отклик'],
            yticklabels=['Нет отклика', 'Отклик'])
plt.title('Матрица ошибок - Random Forest', fontsize=14)
plt.ylabel('Фактическое значение')
plt.xlabel('Предсказанное значение')
plt.show()

# Важность признаков (для Random Forest)
if hasattr(best_rf, 'feature_importances_'):
    # Получаем важность признаков
    feature_importance = pd.DataFrame({
        'feature': all_feature_names,
        'importance': best_rf.feature_importances_
    }).sort_values('importance', ascending=False)

    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance.head(5), x='importance', y='feature', palette='viridis')
    plt.title('Топ-5 наиболее важных признаков (Random Forest)', fontsize=14)
    plt.xlabel('Важность')
    plt.ylabel('Признак')
    plt.tight_layout()
    plt.show()


### МЕТОД 3: GRADIENT BOOSTING

In [ ]:
# Gradient Boosting с подбором гиперпараметров
gb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5]
}

gb = GradientBoostingClassifier(random_state=42)

# Grid Search с кросс-валидацией
gb_grid = GridSearchCV(
    gb, 
    gb_params, 
    cv=StratifiedKFold(5, shuffle=True, random_state=42),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

gb_grid.fit(X_train_processed, y_train)

print("\nЛучшие параметры для Gradient Boosting:")
print(gb_grid.best_params_)
print(f"Лучший ROC-AUC на кросс-валидации: {gb_grid.best_score_:.4f}")

# Лучшая модель
best_gb = gb_grid.best_estimator_

# Предсказания
y_pred_gb = best_gb.predict(X_test_processed)
y_pred_proba_gb = best_gb.predict_proba(X_test_processed)[:, 1]

In [ ]:
# Метрики для Gradient Boosting
print("\n" + "="*60)
print("МЕТРИКИ GRADIENT BOOSTING")
print("="*60)

gb_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_gb),
    'Precision': precision_score(y_test, y_pred_gb),
    'Recall': recall_score(y_test, y_pred_gb),
    'F1-Score': f1_score(y_test, y_pred_gb),
    'ROC-AUC': roc_auc_score(y_test, y_pred_proba_gb)
}

for metric, value in gb_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb, target_names=['Нет отклика', 'Отклик']))

# Матрица ошибок
cm_gb = confusion_matrix(y_test, y_pred_gb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_gb, annot=True, fmt='d', cmap='Oranges', 
            xticklabels=['Нет отклика', 'Отклик'],
            yticklabels=['Нет отклика', 'Отклик'])
plt.title('Матрица ошибок - Gradient Boosting', fontsize=14)
plt.ylabel('Фактическое значение')
plt.xlabel('Предсказанное значение')
plt.show()


In [ ]:
# Сравнение метрик всех моделей
models_comparison = pd.DataFrame({
    'Логистическая регрессия': log_reg_metrics,
    'Random Forest': rf_metrics,
    'Gradient Boosting': gb_metrics
}).T

print("СРАВНЕНИЕ МОДЕЛЕЙ:")
print(models_comparison.round(4))

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

metrics_names = list(log_reg_metrics.keys())
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for idx, metric in enumerate(metrics_names):
    row = idx // 3
    col = idx % 3
    
    values = [log_reg_metrics[metric], rf_metrics[metric], gb_metrics[metric]]
    bars = axes[row, col].bar(['Log Reg', 'Random Forest', 'Gradient Boost'], values, color=colors, edgecolor='black')
    axes[row, col].set_title(f'Сравнение: {metric}', fontsize=12)
    axes[row, col].set_ylim([0, 1])
    axes[row, col].set_ylabel('Значение')
    
    for bar, value in zip(bars, values):
        axes[row, col].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                           f'{value:.3f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Сравнение метрик моделей классификации', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()
# ROC-кривые для всех моделей
plt.figure(figsize=(10, 8))

fpr_log, tpr_log, _ = roc_curve(y_test, y_pred_proba_log_reg)
plt.plot(fpr_log, tpr_log, label=f'Logistic Regression (AUC = {log_reg_metrics["ROC-AUC"]:.3f})', 
         linewidth=2, color='#FF6B6B')

fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {rf_metrics["ROC-AUC"]:.3f})', 
         linewidth=2, color='#4ECDC4')
fpr_gb, tpr_gb, _ = roc_curve(y_test, y_pred_proba_gb)
plt.plot(fpr_gb, tpr_gb, label=f'Gradient Boosting (AUC = {gb_metrics["ROC-AUC"]:.3f})', 
         linewidth=2, color='#45B7D1')

# Диагональная линия
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC-кривые моделей классификации', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### СОХРАНЕНИЕ ЛУЧШЕЙ МОДЕЛИ


In [ ]:
# Определяем лучшую модель по F1-score
model_scores = {
    'Logistic Regression': log_reg_metrics['F1-Score'],
    'Random Forest': rf_metrics['F1-Score'],
    'Gradient Boosting': gb_metrics['F1-Score']
}

best_model_name = max(model_scores, key=model_scores.get)
best_model_score = model_scores[best_model_name]

print(f"Лучшая модель: {best_model_name}")
print(f"F1-Score: {best_model_score:.4f}")

# Сохраняем лучшую модель
if best_model_name == 'Logistic Regression':
    best_model = best_log_reg
elif best_model_name == 'Random Forest':
    best_model = best_rf
else:
    best_model = best_gb

# Сохраняем модель и препроцессор
joblib.dump(best_model, 'best_model.joblib')
joblib.dump(preprocessor, 'preprocessor.pkl')
joblib.dump(all_feature_names, 'feature_names.pkl')

print("\nМодель сохранена как 'best_model.joblib'")
print("Препроцессор сохранен как 'preprocessor.pkl'")
print("Названия признаков сохранены в 'feature_names.pkl'")

In [ ]:
# Функция для загрузки и использования модели
def load_and_predict(model_path, preprocessor_path, new_data):
    """
    Функция для загрузки модели и предсказания на новых данных
    """
    # Загружаем модель и препроцессор
    model = joblib.load(model_path)
    preprocessor = joblib.load(preprocessor_path)
    
    # Обрабатываем новые данные
    new_data_processed = preprocessor.transform(new_data)
    
    # Делаем предсказания
    predictions = model.predict(new_data_processed)
    probabilities = model.predict_proba(new_data_processed)
    
    return predictions, probabilities

print("\nПример использования функции загрузки модели:")
print("predictions, probabilities = load_and_predict('best_model.joblib', 'preprocessor.pkl', new_data)")

In [ ]:
# Пример предсказания на тестовых данных
sample_indices = np.random.choice(len(X_test), 5, replace=False)
sample_data = X_test.iloc[sample_indices]
sample_true = y_test.iloc[sample_indices]

# Предсказания лучшей моделью
sample_pred = best_model.predict(X_test_processed[sample_indices])
sample_proba = best_model.predict_proba(X_test_processed[sample_indices])

print("ПРИМЕР ПРЕДСКАЗАНИЙ НА ТЕСТОВЫХ ДАННЫХ:")
print("-" * 80)
for i in range(len(sample_indices)):
    print(f"Образец {i+1}:")
    print(f"  Фактическое значение: {sample_true.iloc[i]}")
    print(f"  Предсказанное значение: {sample_pred[i]}")
    print(f"  Вероятность отклика: {sample_proba[i][1]:.4f}")
    print(f"  Вероятность отсутствия отклика: {sample_proba[i][0]:.4f}")
    print()

### ИТОГОВЫЕ ВЫВОДЫ

In [ ]:
print("=" * 80)
print("ИТОГОВЫЕ ВЫВОДЫ ПО МОДЕЛЯМ КЛАССИФИКАЦИИ")
print("=" * 80)

print(f"""
На основе проведенного моделирования можно сделать следующие выводы:

1. ЛУЧШАЯ МОДЕЛЬ: {best_model_name}
   - F1-Score: {best_model_score:.4f}
   - ROC-AUC: {model_scores[best_model_name]:.4f}

2. КЛЮЧЕВЫЕ ФАКТОРЫ:
   - Общие расходы клиента (Expenses)
   - Доход (Income)
   - Количество покупок (TotalNumPurchases)
   - Наличие детей (Kids)
   - История участия в кампаниях (TotalAcceptedCmp)

3. РЕКОМЕНДАЦИИ:
   - Фокусироваться на клиентах с высокими расходами
   - Учитывать семейное положение и наличие детей
   - Использовать историю предыдущих кампаний
   - Применять кластеризацию для сегментации

4. ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ:
   - Модель сохранена и готова к использованию
""")

In [ ]:
# Сводная таблица
summary_table = pd.DataFrame({
    'Метод': ['Логистическая регрессия', 'Random Forest', 'Gradient Boosting'],
    'Accuracy': [log_reg_metrics['Accuracy'], rf_metrics['Accuracy'], gb_metrics['Accuracy']],
    'Precision': [log_reg_metrics['Precision'], rf_metrics['Precision'], gb_metrics['Precision']],
    'Recall': [log_reg_metrics['Recall'], rf_metrics['Recall'], gb_metrics['Recall']],
    'F1-Score': [log_reg_metrics['F1-Score'], rf_metrics['F1-Score'], gb_metrics['F1-Score']],
    'ROC-AUC': [log_reg_metrics['ROC-AUC'], rf_metrics['ROC-AUC'], gb_metrics['ROC-AUC']]
})

print("\nСВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:")
print(summary_table.round(4).to_string(index=False))

# Сохраняем результаты
summary_table.to_csv('model_results.csv', index=False)
print("\nРезультаты сохранены в 'model_results.csv'")